In [ ]:
pip install selenium
pip install webdriver_manager
pip install webdriver

In [40]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [41]:

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(" ", "+")
    url = template + '&page={}'
    return url.format(search_term, '{}')




In [42]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    new_url = 'https://www.amazon.com' + atag.get('href')

    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return

    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ""
        review_count = ""

    result = (description, price, rating, review_count, new_url)

    return result


In [43]:
def main():
    from webdriver_manager.chrome import ChromeDriverManager
    driver = webdriver.Chrome(ChromeDriverManager().install())

    search_term = input('Enter the Keyword:')
    record = []

    url = get_url(search_term)

    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            u = extract_record(item)
            if u:
                record.append(u)

    driver.close()

    with open("Productlists.csv", 'w', newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'URL'])
        writer.writerows(record)

    print('Done')

# Call the main function to execute the code
main()


/var/folders/r0/p8v4mr5s7h52zmndncz336vh0000gn/T/ipykernel_39095/1358841838.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Enter the Keyword:perfume
Done
